# [hy] 네이버 카페_레몬테라스 크롤러 with 키워드 입력

### install selenium

In [ ]:
# %pip install selenium==4.1.0
# %pip install webdriver-manager

### 라이브러리 import

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
from webdriver_manager.chrome import ChromeDriverManager

### 검색 키워드 입력

In [ ]:
sample = ['엽떡', '신전']

### 레몬테라스 로그인
##### 크롬드라이버 위치 변경 !

In [ ]:
# 크롬 드라이버 제어
driver = webdriver.Chrome() #현재 컴퓨터 크롬드라이버 위치로 변경

# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login'
driver.get(login_url)
driver.implicitly_wait(5)

# 아이디& 비밀번호 입력
my_id = '본인id' # "id" 대신에 자신의 네이버 아이디 입력
my_pw = '본인password' # "password" 대신에 자신의 네이버 비밀번호 입력

# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
time.sleep(random.uniform(1,1.7))
url = 'https://cafe.naver.com/remonterrace' # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)


### 크롤링할 카페 접속 및 원하는 게시판 클릭 & 게시물 크롤링

In [ ]:
6# 제목, 본문, 댓글, 날짜, 검색 키워드, 게시판 이름 - 빈 list 생성
titles = []
reviews = []
comments = []
dates = []
search_keywords = []
board_titles = []

# board = ['//*[@id="menuLink150"]'] # 크롤링할 게시판의 xpath 입력; 원하는 게시판 클릭 -> F12 눌러 게시판 선택자 확인 가능

# 네이버 카페 접속
# 로그인한 네이버 계정이 해당 카페-게시판에 접근 권한이 있어야 합니다.(회원가입 & 등업)
# 프레임 전환(네이버 카페는 여러 프레임으로 구성되어서 크롤링할 게시글이 있는 프레임으로 전환해주어야 함)

time.sleep(random.uniform(1,1.7))
url = 'https://cafe.naver.com/remonterrace' # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)


time.sleep(1)
for k in sample:
    print(k,"키워드 크롤링 중입니다")
#     for i in board:
#         #원하는 게시판 클릭
#         driver.find_element_by_xpath('%s'%i).click()
#         time.sleep(1)


    #검색
#     driver.switch_to.default_content()
    search_box = driver.find_element(By.XPATH, '//*[@id="topLayerQueryInput"]')
    search_box.send_keys(k)
    search_box.send_keys(Keys.RETURN)
    time.sleep(1)

    driver.switch_to.frame('cafe_main')
    # 3초간 대기 후 다음 코드 실행
    #검색결과 없을 때 다음 게시판으로 넘어가도록

#         데이터 수집기간 : 2022년 1월 ~ 2023년 7월(현재) : 19개월
    time_box = driver.find_element(By.XPATH, '//*[@id="currentSearchDateTop"]')
    time_box.click()
    start_time = driver.find_element(By.XPATH, '//*[@id="input_1_top"]')
    start_time.click()
    time.sleep(0.5)
    start_time.send_keys('20220101')
    time.sleep(1)
    end_time = driver.find_element(By.XPATH, '//*[@id="input_2_top"]')
    end_time.click()
    time.sleep(0.5)
    end_time.send_keys('20240430')
    driver.find_element(By.XPATH, '//*[@id="btn_set_top"]').click()
    time.sleep(1)




    driver.find_element(By.XPATH, '//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button').click()



    try:
        if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
            driver.get(url)
            time.sleep(1)
            continue

    except:
        print("크롤링 시작합미당~")
        pass

    #driver.switch_to.frame('cafe_main') # 프레임 전환

    # 50개씩 보기
    driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]').click()
    driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

    #url 가져오기
    page_url_list = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.article')]
    first_page = driver.find_element(By.CSS_SELECTOR,'a.on').get_attribute('href')
    #페이지 넘기기 검색시 최대 100페이지
    for j in range(1,41):
        #400개마다 껐다 키기
        try:
            if j % 8 == 0:
                #다음 링크 가져오기
                time.sleep(1)
                link = first_page[:-1] + str(j)


                driver.quit()
                driver = webdriver.Chrome() #현재 컴퓨터 크롬드라이버 위치로 변경


                ##재로그인
                # 네이버 로그인 화면 이동
                login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
                driver.get(login_url)
                driver.implicitly_wait(10)


                driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
                driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
                time.sleep(1)


                # '로그인' 버튼 클릭
                driver.find_element('id', 'log.login').click()
                time.sleep(1)

                #다음 링크부터 가져오기
                driver.get(link)
                driver.switch_to.frame('cafe_main')

        except:
            pass
            #결과 없을 때 넘어가기
        try:
            if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
                break
        except:
            pass
        if j == 1:
            pass
        else:
            next_page = first_page[:-1] + str(j)
            driver.get(next_page)
            time.sleep(1)
            driver.switch_to.frame('cafe_main')

        page_url_list = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.article')]


        for url in page_url_list:
            try:
                driver.get(url)
                time.sleep(random.uniform(1,1.7))
                driver.switch_to.frame('cafe_main')
            except :
                continue
            try:

                #게시판 이름 수집
                board_title = driver.find_element(By.CSS_SELECTOR, 'div.ArticleTitle').text
                board_titles.append(board_title)
                title = driver.find_element(By.CSS_SELECTOR, 'h3.title_text').text
                titles.append(title)

                #날짜 수집
                day = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div.profile_area > div.article_info > span.date').text
                soup = bs(driver.page_source, 'lxml')
                isdate = soup.find_all('span',class_='date')
                dates.append(isdate)

                #키워드 수집
                search_keywords.append(k)

            except:
                continue
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text').text
                reviews.append(content)
            except:
                try:
                    content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer').text
                    reviews.append(content)
                except:
                    try:
                        content = driver.find_element(By.CSS_SELECTOR, 'div.scrap_added').text.strip()
                        reviews.append(content)
                    except:
                        content= [] #게시물 본문이 사진으로만 구성되어있을때
                        reviews.append(content)

            # 댓글 수집
            soup = bs(driver.page_source, 'lxml')
            iscomment = soup.find_all('span',class_='text_comment')
            if len(iscomment) == 0:
                comment = '댓글 없음'

            else:
                WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "text_comment")) )
#                 soup = bs(driver.page_source, 'lxml')
                iscomment = soup.find_all('span',class_='text_comment')
                comment = ''
                for i in iscomment:
                    comment += i.get_text() + '. '

            comments.append(comment)
#             comments.append(box)


    url = 'https://cafe.naver.com/remonterrace' # 크롤링할 카페 url 입력
    driver.get(url)
    time.sleep(1)

# # 크롬 창 닫기
driver.quit()

### 클롤링 개수 확인(키워드, 게시판, 제목, 본문, 댓글, 날짜)

In [ ]:
print(len(search_keywords))
print(len(board_titles))
print(len(titles))
print(len(reviews))
print(len(comments))
print(len(dates))

In [ ]:
data = {'keyword' : search_keywords,
#         'board' : formatted_boards,
        'title' : titles,
        'body' : reviews,
        'comment' : comments,
        'date': dates}

dataDF = pd.DataFrame(data)

### 크롤링 데이터 합치기

In [ ]:
#날짜 포멧 '2023.03.20.' 위 형태로 포맷
import re

formatted_dates = []
#게시판 제목 포맷
formatted_boards = []
for board_title in board_titles:
    formatted_boards.append(board_title.split("\n")[0])


for date in [str(d[0]) for d in dates]:
    date_str = re.search(r'\d{4}\.\d{2}\.\d{2}\.', date).group(0)
    formatted_dates.append(date_str)

# ===데이터 합치기===

data = {'keyword' : search_keywords,
        'board' : formatted_boards,
        'title' : titles,
        'body' : reviews,
        'comment' : comments,
        'date': formatted_dates}

dataDF = pd.DataFrame(data)

### CSV 저장 & 데이터 확인

In [ ]:
dataDF

In [ ]:
dataDF

#csv로 저장
dataDF.to_csv("[hy]대분류이름_담당자.csv", encoding = "utf-8-sig") #파일 이름 변경 필요!

- 누가 크롤링한 건지 확인할 수 있도록 키워드, 담당자 이름을 꼭 파일명에 넣어주세요!
- 어떤 키워드로 검색했을 때 나온 결과인지 알 수 있도록 키워드 꼬옥 넣어주세요!